In [1]:
import os
import pandas as pd
from shutil import copy2

def convert_to_yolo(row, width, height):
    x1 = row['Roi.X1']
    y1 = row['Roi.Y1']
    x2 = row['Roi.X2']
    y2 = row['Roi.Y2']

    box_width = x2 - x1
    box_height = y2 - y1
    center_x = x1 + box_width / 2
    center_y = y1 + box_height / 2

    center_x /= width
    center_y /= height
    box_width /= width
    box_height /= height

    return center_x, center_y, box_width, box_height

def process_csv(csv_path, images_root_dir, output_images_dir, output_labels_dir):
    data_csv = pd.read_csv(csv_path)
    for index, row in data_csv.iterrows():
        image_path = row['Path']
        image_name = os.path.basename(image_path)
        txt_name = image_name.replace('.png', '.txt')

        width, height = row['Width'], row['Height']
        center_x, center_y, box_width, box_height = convert_to_yolo(row, width, height)
        cls = row['ClassId']

        src = os.path.join(images_root_dir, image_path)
        dst = os.path.join(output_images_dir, image_name)

        os.makedirs(output_images_dir, exist_ok=True)

        if os.path.exists(src):
            copy2(src, dst)
        else:
            continue

        label_path = os.path.join(output_labels_dir, txt_name)
        os.makedirs(output_labels_dir, exist_ok=True)
        with open(label_path, 'w') as f:
            f.write(f'{cls} {center_x} {center_y} {box_width} {box_height}\n')

train_csv_path = '../input/gtsrb-german-traffic-sign/Train.csv'
test_csv_path = '../input/gtsrb-german-traffic-sign/Test.csv'
images_root_dir = '../input/gtsrb-german-traffic-sign'  
output_train_images_dir = 'datasets/train/images'
output_train_labels_dir = 'datasets/train/labels'
output_test_images_dir = 'datasets/val/images'
output_test_labels_dir = 'datasets/val/labels'

process_csv(train_csv_path, images_root_dir, output_train_images_dir, output_train_labels_dir)
process_csv(test_csv_path, images_root_dir, output_test_images_dir, output_test_labels_dir)


In [2]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [3]:

!yolo detect train data=/kaggle/input/datasetyml/data.yml model=yolov8s.yaml epochs=10 imgsz=640 batch=64


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256